In [1]:
!pip install py_vncorenlp
!pip install faiss-cpu
!pip install rank_bm25
!pip install pyvi
!pip install vncorenlp
!pip install sentence_transformers

ERROR: Could not find a version that satisfies the requirement py_vncorenlp (from versions: none)
ERROR: No matching distribution found for py_vncorenlp
ERROR: Could not find a version that satisfies the requirement faiss-cpu (from versions: none)
ERROR: No matching distribution found for faiss-cpu
ERROR: Could not find a version that satisfies the requirement rank_bm25 (from versions: none)
ERROR: No matching distribution found for rank_bm25
ERROR: Could not find a version that satisfies the requirement pyvi (from versions: none)
ERROR: No matching distribution found for pyvi
ERROR: Could not find a version that satisfies the requirement vncorenlp (from versions: none)
ERROR: No matching distribution found for vncorenlp
ERROR: Could not find a version that satisfies the requirement sentence_transformers (from versions: none)
ERROR: No matching distribution found for sentence_transformers


In [ ]:
%cp /kaggle/input/kalapa-challenge-medical-processing-data/Processed_Data/nlp_processing.py /kaggle/working/

In [ ]:
PATH_PUBLIC_TEST = "/kaggle/input/kalapa-medical-challenge/MEDICAL_kalapa/public_test.csv"
PATH_CLEANED_CORPUS = "/kaggle/input/kalapa-challenge-medical-processing-data/Processed_Data/save_cleaned_info_corpus_v2.pkl"
PATH_CORPUS = "/kaggle/input/kalapa-medical-challenge/MEDICAL_kalapa/corpus"


In [ ]:
from bs4 import BeautifulSoup
import os 
import numpy as np 
from tqdm import tqdm 
import time 
import re
import matplotlib.pyplot as plt
import numpy as np
import faiss 
from faiss import write_index, read_index
import pandas as pd
from rank_bm25 import BM25Okapi
import torch
from nlp_processing import tien_xu_li
import pickle 

In [ ]:
##test vncorenlp
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("/kaggle/input/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g')
vncorenlp_example = vncorenlp.tokenize("VTV đồng ý chia sẻ tiền bản quyền World Cup 2022 để khai thác.")
print(vncorenlp_example)


In [ ]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

In [ ]:
def clean_answer(answer):
    l_cleaned = ["A:", "B:", "C:", "D:", "E:", "F:", "A.", "B.", "C.", "D.", "E.", "F."]
    for item in l_cleaned:
        answer = answer.replace(item, "")
    answer = " ".join(answer.split())
    return answer

In [ ]:
file_cleaned_corpus =  open(PATH_CLEANED_CORPUS, 'rb')
l_cleaned_corpus = pickle.load(file_cleaned_corpus)

In [ ]:
def prepare_sentece_for_encode(corpus):
    title = corpus["title"]
    content = corpus["content"]
    l_sub_titles = corpus["l_sub_titles"]
    l_sub_contents = corpus["l_sub_contents"]
    sub_corpus = []
    for idx in range(len(l_sub_titles)):
        sentences = title + ": " + content + "## " + l_sub_titles[idx] + ": " + l_sub_contents[idx]
        sentences = sentences.replace("\r", "").replace("\n", "").replace("\t", "")
        sentences = " ".join(sentences.split())
        sub_corpus.append(sentences)
    return sub_corpus

In [ ]:
l_sub_corpus = []
l_idx_corpus = []
for idx, corpus in tqdm(enumerate(l_cleaned_corpus)):
    sub_corpus = prepare_sentece_for_encode(corpus)
    l_sub_corpus += sub_corpus
    l_idx_corpus += [idx for item in sub_corpus]

In [ ]:
print(len(l_sub_corpus))

In [ ]:
tokenized_corpus = [tien_xu_li(doc).split(" ") for doc in l_sub_corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
#Load public test.csv and search 
df = pd.read_csv(PATH_PUBLIC_TEST)
l_questions = df["question"].tolist()
l_answer_per_questions = []
l_columns = ['option_1','option_2','option_3','option_4','option_5','option_6']
for index, row in df.iterrows():
    l_answers = []
    for column in l_columns: 
        if not isinstance(row[column], str):
            continue 
        else:
            l_answers.append(clean_answer(row[column]))
    l_answer_per_questions.append(l_answers)

In [ ]:
def calculate_cosince_similarity(text_1, text_2):
    text_1 = tien_xu_li(text_1)
    text_2 = tien_xu_li(text_2)
    sentences_token_1 = [tokenize(sentence) for sentence in [text_1]]
    embedding_1 = model.encode(sentences_token_1)
    embedding_1 = torch.from_numpy(embedding_1)
    sentences_token_2 = [tokenize(sentence) for sentence in [text_2]]
    embedding_2 = model.encode(sentences_token_2)
    embedding_2 = torch.from_numpy(embedding_2)
    rs = torch.cosine_similarity(embedding_1.reshape(1,-1), embedding_2.reshape(1,-1))
    rs = rs.numpy()[0]
    return rs

In [ ]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
def select_top_1_answer(answer_per_question, question, relavant_corpus):
    l_prediction = ['0' for item in answer_per_question]
    l_cosine_scores = []
    for answer in answer_per_question:
        cosine_score = calculate_cosince_similarity(question + " ## " + answer, relavant_corpus)
        l_cosine_scores.append(cosine_score)
    idx_seletect = np.argmax(np.array(l_cosine_scores))
    l_prediction[idx_seletect] = '1'
    return l_prediction

In [ ]:
'''
là gì -> select 1 
không -> select 1 
đúng hay sai -> select 1
bằng cách gì -> select 1 

'''

def get_predict_1_answer(answer_per_question, question, relavant_corpus):
    l_prediction = ['0' for item in answer_per_question]
    l_cosine_scores = []
    for answer in answer_per_question:
        cosine_score = calculate_cosince_similarity(answer, relavant_corpus)
        l_cosine_scores.append(cosine_score)
    idx_seletect = np.argmax(np.array(l_cosine_scores))
    l_prediction[idx_seletect] = '1'
    prediction_str = "".join(l_prediction)
    return prediction_str

def get_predict_true_false(answer_per_question, question, relavant_corpus):
    l_prediction = ['0' for item in answer_per_question]
    cosine_score = calculate_cosince_similarity(question, relavant_corpus)
    if cosine_score > 0.5:
        l_prediction[0] = '1'
    else:
        l_prediction[-1] = '1'
    prediction_str = "".join(l_prediction)
    return prediction_str

def get_predict_multi_answer(answer_per_question, question, relavant_corpus):
    THRESHOLD = 0.5
    l_prediction = ['0' for item in answer_per_question]
    found_matched = False
    if "Có" not in answer_per_question and "có" not in answer_per_question:
        #change other method 
        for idx_answer, answer in enumerate(answer_per_question):
            tmp_answer = vncorenlp.tokenize(answer)
            keywords_answers = []
            for sentence in tmp_answer:
                keywords_answers += [word.lower() for word in sentence]
            #split relavant corpus 
            keywords_corpus = []
            tmp_corpus = vncorenlp.tokenize(relavant_corpus)
            for sentence in tmp_corpus:
                keywords_corpus += [word.lower() for word in sentence]
            l_overlap = intersection(keywords_answers, keywords_corpus)
            ratio_overlap = len(l_overlap) / len(keywords_answers)
            if ratio_overlap > THRESHOLD:
                l_prediction[idx_answer] = '1'
                found_matched = True
    if not found_matched:
        l_prediction = select_top_1_answer(answer_per_question, question, relavant_corpus)
    prediction_str = "".join(l_prediction)
    return prediction_str

In [ ]:
def get_predict(answer_per_question, question, relavant_corpus):
    l_1_answers = ["là gì", "cách gì"]
    l_true_false = ["đúng hay sai", "không"]
    type_predict = 3
    if "là gì" in question and "các" not in question:
        type_predict = 1
    if "cách gì" in question and "các" not in question:
        type_predict = 1
    if "đúng hay sai" in question:
        type_predict = 2
    if "không" in question:
        type_predict = 2            
    if type_predict == 1:
        prediction_str = get_predict_1_answer(answer_per_question, question, relavant_corpus)
    elif type_predict == 2:
        prediction_str = get_predict_true_false(answer_per_question, question, relavant_corpus)
    else:
        prediction_str = get_predict_multi_answer(answer_per_question, question, relavant_corpus)
    return prediction_str

In [ ]:
def find_relavant_in_current_document(idx_matched_info, l_infos, question, answer_per_question):
    index_faiss = faiss.IndexFlatL2(768)
    info = l_infos[idx_matched_info]
    subcorpus = prepare_sentece_for_encode(info)
    sentences = [tien_xu_li(item) for item in subcorpus]
    sentences_token = [tokenize(sentence) for sentence in sentences]
    embedding = model.encode(sentences_token)
    embedding = np.array(embedding).astype('float32')
    index_faiss.add(embedding)
    
    question_with_answer = question
    for answer in answer_per_question:
        question_with_answer += " " + answer
    question_with_answer = tien_xu_li(question_with_answer)
    sentences_token_qa = [tokenize(sentence) for sentence in [question_with_answer]]
    embedding_qa = model.encode(sentences_token_qa)[0] 
    embedding_qa = np.array([embedding_qa]).astype('float32')
    f_dists, f_ids = index_faiss.search(embedding_qa.reshape(1, -1), k=1)
    return subcorpus[f_ids[0][0]]

In [ ]:
def get_top_1_embedding(question, l_raw_sentences_matched):
    index_faiss = faiss.IndexFlatL2(768)
    sentences = [tien_xu_li(item) for item in l_raw_sentences_matched]
    sentences_token = [tokenize(sentence) for sentence in sentences]
    embedding = model.encode(sentences_token)
    embedding = np.array(embedding).astype('float32')
    index_faiss.add(embedding)
    
    
    sentences_token_qa = [tokenize(sentence) for sentence in [question]]
    embedding_qa = model.encode(sentences_token_qa)[0] 
    embedding_qa = np.array([embedding_qa]).astype('float32')


    f_dists, f_ids = index_faiss.search(embedding_qa.reshape(1, -1), k=1)
    return f_ids[0][0]
    

In [ ]:
l_predict_df = []
for idx_qa, question in enumerate(l_questions):
    question = tien_xu_li(question)
    tokenized_query = question.split(" ")
    l_idxs = [i for i in range(len(l_sub_corpus))]
    search_idx_selected = bm25.get_top_n(tokenized_query,l_idxs , n=5)
    answer_per_question = l_answer_per_questions[idx_qa]
    l_subcorpus_matched = [l_sub_corpus[idx] for idx in search_idx_selected]
    embedding_selected_idx = get_top_1_embedding(question, l_subcorpus_matched)
    idx_corpus_matched = l_idx_corpus[search_idx_selected[embedding_selected_idx]]
    selected_sentences = find_relavant_in_current_document(idx_corpus_matched, l_cleaned_corpus,\
                                                           question, answer_per_question)
    prediction_str = get_predict(answer_per_question, question, selected_sentences)
    l_predict_df.append(prediction_str)
    
#     if prediction_str != l_labels[idx_qa]:
    print(answer_per_question)
    print("Question: ", question)
    print("SELECTED CORPUS: ", selected_sentences)
    print("PREDICTION: ", prediction_str)
    print("#########################")

In [ ]:
#create submission with full 1
df_public = pd.read_csv(PATH_PUBLIC_TEST)
l_id = df_public["id"].tolist()

df_submit = pd.DataFrame()
df_submit["id"] = l_id
df_submit["answer"] = l_predict_df

df_submit.to_csv("submission.csv", index = False)